# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f49e088fe50>
├── 'a' --> tensor([[-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644]])
└── 'x' --> <FastTreeValue 0x7f49e088ffa0>
    └── 'c' --> tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                        [ 0.9165,  0.7652,  0.1698,  0.6468],
                        [-1.2986, -0.4070,  0.4114,  0.4565]])

In [4]:
t.a

tensor([[-0.3880,  1.0906,  0.3177],
        [-2.8200, -0.3253,  0.2644]])

In [5]:
%timeit t.a

56.9 ns ± 0.0574 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f49e088fe50>
├── 'a' --> tensor([[-0.5525, -1.5487, -0.4586],
│                   [-0.4512,  2.4262, -1.8614]])
└── 'x' --> <FastTreeValue 0x7f49e088ffa0>
    └── 'c' --> tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                        [ 0.9165,  0.7652,  0.1698,  0.6468],
                        [-1.2986, -0.4070,  0.4114,  0.4565]])

In [7]:
%timeit t.a = new_value

71.8 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644]]),
    x: Batch(
           c: tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                      [ 0.9165,  0.7652,  0.1698,  0.6468],
                      [-1.2986, -0.4070,  0.4114,  0.4565]]),
       ),
)

In [10]:
b.a

tensor([[-0.3880,  1.0906,  0.3177],
        [-2.8200, -0.3253,  0.2644]])

In [11]:
%timeit b.a

50.7 ns ± 0.0306 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6465,  0.0154, -0.4286],
               [ 0.3726, -0.1822,  1.6612]]),
    x: Batch(
           c: tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                      [ 0.9165,  0.7652,  0.1698,  0.6468],
                      [-1.2986, -0.4070,  0.4114,  0.4565]]),
       ),
)

In [13]:
%timeit b.a = new_value

505 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

9.84 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

134 µs ± 6.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f49e0822730>
├── 'a' --> tensor([[[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]],
│           
│                   [[-0.3880,  1.0906,  0.3177],
│                    [-2.8200, -0.3253,  0.2644]]])
└── 'x' --> <FastTreeValue 0x7f49e0822670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f49e0819790>
├── 'a' --> tensor([[-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644],
│                   [-0.3880,  1.0906,  0.3177],
│                   [-2.8200, -0.3253,  0.2644]])
└── 'x' --> <FastTreeValue 0x7f49e0819370>
    └── 'c' --> tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                        [ 0.9165,  0.7652,  0.1698,  0.6468],
                 

In [23]:
%timeit t_cat(trees)

28 µs ± 56.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]],
       
               [[-0.3880,  1.0906,  0.3177],
                [-2.8200, -0.3253,  0.2644]]]),
    x: Batch(
           c: tensor([[[-0.1465, -1.3721, -0.4334,  0.9829],
                       [ 0.9165,  0.7652,  0.1698,  0.6468],
                       [-1.2986, -0.4070,  0.4114,  0.4565]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644],
               [-0.3880,  1.0906,  0.3177],
               [-2.8200, -0.3253,  0.2644]]),
    x: Batch(
           c: tensor([[-0.1465, -1.3721, -0.4334,  0.9829],
                      [ 0.9165,  0.7652,  0.1698,  0.6468],
                      [-1.2986, -0.4070,  0.4114,  0.4565],
                      [-0.1465, -1.3721, -0.4334,  0.9829],
                      [ 0.9165,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 344 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

293 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
